In [135]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import pandas as pd
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../plant_pathology/scripts/')
# from dataset.dataset import read_csv, ReadCSV
import os
from sklearn.preprocessing import LabelEncoder
from train.loss import loss_fn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [107]:
temp = ReadCSV('../../plant_path_data/train.csv', '../../plant_path_data/train_images/')

In [108]:
temp.read_csv()
temp.check_paths()
temp.encode_labels()
temp.one_hot_encoder()
temp.split_datasets()

In [109]:
data = temp.get_object()

In [111]:
data.check_image_valids(cpu_count=22)

In [43]:
temp['labels'].values

array(['healthy', 'scab frog_eye_leaf_spot complex', 'scab', ..., 'rust',
       'scab frog_eye_leaf_spot', 'healthy'], dtype=object)

In [47]:
model = LabelEncoder().fit_transform(temp['labels'].values)

In [114]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

In [115]:
input

tensor([[ 1.3511, -0.6018,  0.1149,  0.5135, -1.3915],
        [ 0.9774,  0.9683,  1.1193, -0.6308,  0.1796],
        [-1.5151,  0.2824, -1.3485, -1.1353,  1.7429]], requires_grad=True)

In [121]:
input = torch.randn(3, 5)
target = torch.empty(3, dtype=torch.long).random_(5)